In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [5]:
bets = pd.read_csv('In_Season/Data/todays_bets.csv', index_col = 0)
# bets.drop([6], inplace = True)
# bets.reset_index(inplace = True, drop = True)
# bets.to_csv('In_Season/Data/todays_bets.csv')

In [7]:
results = pd.read_csv('In_Season/Data/results_tracker.csv', index_col = 0)
# results = results[results.Date != '2022-02-15']
# results.to_csv('In_Season/Data/results_tracker.csv')

In [19]:
external_bets = pd.read_csv('In_Season/Data/todays_bets_external.csv')

In [24]:
# Initializing necessary objects
yesterday_bets = pd.read_csv('In_Season/Data/todays_bets_external.csv', index_col = 0)
yesterday_bets.head()

,Date,Away_Team,Away_Prob,Home_Team,Home_Prob,Home_Odds,Away_Odds,Home_Prob_Implied,Away_Prob_Implied,Home_Prob_Diff,...,Away_Prob_Adjusted,Home_Prob_Combined,Away_Prob_Combined,Home_Prob_Diff_Combined,Away_Prob_Diff_Combined,Home_Bet_Size_Combined,Away_Bet_Size_Combined,Home_Bet_Combined,Away_Bet_Combined,Potential_Payoff_Combined
0,2022-02-16,Atlanta Hawks,0.62,Orlando Magic,0.38,190.0,-230.0,0.344828,0.696970,0.035172,...,0.659763,0.360118,0.639882,0.015291,-0.057088,0.002334,0.0,233.387385,0.0,443.436032
1,2022-02-16,Brooklyn Nets,0.37,New York Knicks,0.63,-190.0,160.0,0.655172,0.384615,-0.025172,...,0.121027,0.754486,0.245514,0.099314,-0.139102,0.028801,0.0,2880.102016,0.0,1515.843166
2,2022-02-16,Detroit Pistons,0.07,Boston Celtics,0.93,-900.0,610.0,0.900000,0.140845,0.030000,...,0.054719,0.937641,0.062359,0.037641,-0.078486,0.037641,0.0,3764.062069,0.0,418.229119
3,2022-02-16,Washington Wizards,0.34,Indiana Pacers,0.66,-120.0,102.0,0.545455,0.495050,0.114545,...,0.442761,0.608619,0.391381,0.063165,-0.103669,0.013896,0.0,1389.624717,0.0,1158.020597
4,2022-02-16,Portland Trailblazers,0.15,Memphis Grizzlies,0.85,-700.0,500.0,0.875000,0.166667,-0.025000,...,0.066170,0.891915,0.108085,0.016915,-0.058582,0.013532,0.0,1353.215388,0.0,193.316484


In [21]:
# Getting winners (not necessary in VS Code)

team_map_results = {
    'LAL' : 'Los Angeles Lakers',
    'LAC' : 'Los Angeles Clippers',
    'POR' : 'Portland Trailblazers',
    'CLE' : 'Cleveland Cavaliers',
    'DEN' : 'Denver Nuggets',
    'DAL' : 'Dallas Mavericks',
    'UTA' : 'Utah Jazz',
    'SA' : 'San Antonio Spurs',
    'ATL' : 'Atlanta Hawks',
    'CHA' : 'Charlotte Hornets',
    'CHI' : 'Chicago Bulls',
    'DET' : 'Detroit Pistons',
    'MIL' : 'Milwaukee Bucks',
    'ORL' : 'Orlando Magic',
    'MIN' : 'Minnesota Timberwolves',
    'PHO' : 'Phoenix Suns',
    'BOS' : 'Boston Celtics',
    'IND' : 'Indiana Pacers',
    'SAC' : 'Sacramento Kings',
    'TOR' : 'Toronto Raptors',
    'WAS' : 'Washington Wizards',
    'BKN' : 'Brooklyn Nets',
    'NO' : 'New Orleans Pelicans',
    'DAL' : 'Dallas Mavericks',
    'PHI' : 'Philadelphia 76ers',
    'MIA' : 'Miami Heat',
    'MEM' : 'Memphis Grizzlies',
    'GS' : 'Golden State Warriors',
    'NY' : 'New York Knicks',
    'HOU' : 'Houston Rockets',
    'OKC' : 'Oklahoma City Thunder'
}

# Instantiating selenium driver
link = 'https://www.cbssports.com/nba/schedule/'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)

# Getting table for yesterday's games and formatting
day_buttons = driver.find_elements_by_class_name("ToggleContainer-text")
yesterday = day_buttons[0]
yesterday.click()
WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='TableBase']/div/div/table")))
html_yesterday = driver.page_source
tables_yesterday = pd.read_html(html_yesterday)
try:
    yesterday_schedule = tables_yesterday[1]
except:
    yesterday_schedule = tables_yesterday[0]
yesterday_schedule = yesterday_schedule[['Away', 'Home', 'Result']]

# Populating yesterday's schedule w/ winner
yesterday_schedule['Winner'] = ''
for index, row in yesterday_schedule.iterrows():
    winner = row.Result[:3]
    yesterday_schedule.loc[index, 'Winner'] = winner
yesterday_schedule['Winner'] = yesterday_schedule.Winner.str.strip(' ')
yesterday_schedule['Winner'] = yesterday_schedule.Winner.apply(lambda x: team_map_results[x])

# Determining if bet hit and calculating running total of capital
winners = list(yesterday_schedule.Winner.unique())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache
/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/hsinger

In [29]:
capital_538 = 100000
capital_combined = 100000

# Determining if bet hit and calculating running total of capital

yesterday_bets['Won_Bet_538'] = -1 
yesterday_bets['Won_Bet_Combined'] = -1 
yesterday_bets['Capital_538'] = capital_538
yesterday_bets['Capital_Combined'] = capital_combined
for index, row in yesterday_bets.iterrows():
    if index == 0:
        
        # 538
        if row.Home_Bet > 0:
            if row.Home_Team in winners:
                yesterday_bets.loc[index, 'Won_Bet_538'] = 1
                yesterday_bets.loc[index, 'Capital_538'] = capital_538 + row.Potential_Payoff
            else:
                yesterday_bets.loc[index, 'Won_Bet_538'] = 0
                yesterday_bets.loc[index, 'Capital_538'] = capital_538 - row.Home_Bet
        if row.Away_Bet > 0:
            if row.Away_Team in winners:
                yesterday_bets.loc[index, 'Won_Bet_538'] = 1
                yesterday_bets.loc[index, 'Capital_538'] = capital_538 + row.Potential_Payoff
            else:
                yesterday_bets.loc[index, 'Won_Bet_538'] = 0
                yesterday_bets.loc[index, 'Capital_538'] = capital_538 - row.Away_Bet
        
        # Combined
        if row.Home_Bet_Combined > 0:
            if row.Home_Team in winners:
                yesterday_bets.loc[index, 'Won_Bet_Combined'] = 1
                yesterday_bets.loc[index, 'Capital_Combined'] = capital_combined + row.Potential_Payoff_Combined
            else:
                yesterday_bets.loc[index, 'Won_Bet_Combined'] = 0
                yesterday_bets.loc[index, 'Capital_Combined'] = capital_combined - row.Home_Bet_Combined
        if row.Away_Bet_Combined > 0:
            if row.Away_Team in winners:
                yesterday_bets.loc[index, 'Won_Bet_Combined'] = 1
                yesterday_bets.loc[index, 'Capital_Combined'] = capital_combined + row.Potential_Payoff_Combined
            else:
                yesterday_bets.loc[index, 'Won_Bet_Combined'] = 0
                yesterday_bets.loc[index, 'Capital_Combined'] = capital_combined - row.Away_Bet_Combined
    else:
        
        # 538
        if row.Potential_Payoff == 0:
            yesterday_bets.loc[index, 'Capital_538'] = yesterday_bets.loc[(index - 1), 'Capital_538']
        if row.Home_Bet > 0:
            if row.Home_Team in winners:
                yesterday_bets.loc[index, 'Won_Bet_538'] = 1
                yesterday_bets.loc[index, 'Capital_538'] = yesterday_bets.loc[(index - 1), 'Capital_538'] + row.Potential_Payoff
            else:
                yesterday_bets.loc[index, 'Won_Bet_538'] = 0
                yesterday_bets.loc[index, 'Capital_538'] = yesterday_bets.loc[(index - 1), 'Capital_538'] - row.Home_Bet
        if row.Away_Bet > 0:
            if row.Away_Team in winners:
                yesterday_bets.loc[index, 'Won_Bet_538'] = 1
                yesterday_bets.loc[index, 'Capital_538'] = yesterday_bets.loc[(index - 1), 'Capital_538'] + row.Potential_Payoff
            else:
                yesterday_bets.loc[index, 'Won_Bet_538'] = 0
                yesterday_bets.loc[index, 'Capital_538'] = yesterday_bets.loc[(index - 1), 'Capital_538'] - row.Away_Bet
        
        # Combined
        if row.Potential_Payoff_Combined == 0:
            yesterday_bets.loc[index, 'Capital_Combined'] = yesterday_bets.loc[(index - 1), 'Capital_Combined']
        if row.Home_Bet_Combined > 0:
            if row.Home_Team in winners:
                yesterday_bets.loc[index, 'Won_Bet_Combined'] = 1
                yesterday_bets.loc[index, 'Capital_Combined'] = yesterday_bets.loc[(index - 1), 'Capital_Combined'] + row.Potential_Payoff_Combined
            else:
                yesterday_bets.loc[index, 'Won_Bet_Combined'] = 0
                yesterday_bets.loc[index, 'Capital_Combined'] = yesterday_bets.loc[(index - 1), 'Capital_Combined'] - row.Home_Bet_Combined
        if row.Away_Bet_Combined > 0:
            if row.Away_Team in winners:
                yesterday_bets.loc[index, 'Won_Bet_Combined'] = 1
                yesterday_bets.loc[index, 'Capital_Combined'] = yesterday_bets.loc[(index - 1), 'Capital_Combined'] + row.Potential_Payoff_Combined
            else:
                yesterday_bets.loc[index, 'Won_Bet_Combined'] = 0
                yesterday_bets.loc[index, 'Capital_Combined'] = yesterday_bets.loc[(index - 1), 'Capital_Combined'] - row.Away_Bet_Combined

In [30]:
yesterday_bets

,Date,Away_Team,Away_Prob,Home_Team,Home_Prob,Home_Odds,Away_Odds,Home_Prob_Implied,Away_Prob_Implied,Home_Prob_Diff,...,Away_Prob_Diff_Combined,Home_Bet_Size_Combined,Away_Bet_Size_Combined,Home_Bet_Combined,Away_Bet_Combined,Potential_Payoff_Combined,Won_Bet_538,Won_Bet_Combined,Capital_538,Capital_Combined
0,2022-02-16,Atlanta Hawks,0.62,Orlando Magic,0.38,190.0,-230.0,0.344828,0.696970,0.035172,...,-0.057088,0.002334,0.000000,233.387385,0.000000,443.436032,0,0,99463.157895,99766.612615
1,2022-02-16,Brooklyn Nets,0.37,New York Knicks,0.63,-190.0,160.0,0.655172,0.384615,-0.025172,...,-0.139102,0.028801,0.000000,2880.102016,0.000000,1515.843166,-1,0,99463.157895,96886.510599
2,2022-02-16,Detroit Pistons,0.07,Boston Celtics,0.93,-900.0,610.0,0.900000,0.140845,0.030000,...,-0.078486,0.037641,0.000000,3764.062069,0.000000,418.229119,1,1,99796.491228,97304.739718
3,2022-02-16,Washington Wizards,0.34,Indiana Pacers,0.66,-120.0,102.0,0.545455,0.495050,0.114545,...,-0.103669,0.013896,0.000000,1389.624717,0.000000,1158.020597,0,0,97276.491228,95915.115001
4,2022-02-16,Portland Trailblazers,0.15,Memphis Grizzlies,0.85,-700.0,500.0,0.875000,0.166667,-0.025000,...,-0.058582,0.013532,0.000000,1353.215388,0.000000,193.316484,-1,1,97276.491228,96108.431485
5,2022-02-16,San Antonio Spurs,0.64,Oklahoma City Thunder,0.36,260.0,-320.0,0.277778,0.761905,0.082222,...,-0.061600,0.003035,0.000000,303.471455,0.000000,789.025783,0,0,96138.029690,95804.960030
6,2022-02-16,Sacramento Kings,0.38,Chicago Bulls,0.62,-270.0,220.0,0.729730,0.312500,-0.109730,...,-0.030524,0.000000,0.000000,0.000000,0.000000,0.000000,0,-1,95156.211508,96138.029690
7,2022-02-16,Toronto Raptors,0.40,Minnesota Timberwolves,0.60,102.0,-120.0,0.495050,0.545455,0.104950,...,-0.128868,0.017500,0.000000,1749.954278,0.000000,1784.953364,1,1,97276.211508,97922.983054
8,2022-02-16,Houston Rockets,0.09,Phoenix Suns,0.91,-1350.0,810.0,0.931034,0.109890,-0.021034,...,-0.050585,0.014008,0.000000,1400.776042,0.000000,103.761188,-1,1,97276.211508,98026.744242
9,2022-02-16,Denver Nuggets,0.33,Golden State Warriors,0.67,-215.0,180.0,0.682540,0.357143,-0.012540,...,0.123573,0.000000,0.019222,0.000000,1922.246294,3460.043330,-1,0,97276.211508,96104.497948
